Langkah 1 - Import Library

In [14]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Langkah 2 - Pra Pengolahan Data

Langkah 2.1. Pra Pengolahan Data Training

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# melengkapi data pelatihan agar lebih kuat terhadap variasi dalam data input
# mengubah rentang nilai piksel menjadi 0-1
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   # memberikan variasi pada gambar dengan melakukan shear dan zoom
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   # melakukan flipping horizontal.
                                   horizontal_flip = True)
# generator yang menghasilkan batch data pelatihan
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8010 images belonging to 2 classes.


Langkah 2.2. Pra Pengolahan Data Testing

In [17]:
# Membuat objek ImageDataGenerator untuk data uji tanpa augmentasi.
# Rescale digunakan untuk mengubah rentang nilai piksel menjadi 0-1, tetapi tidak ada operasi augmentasi lainnya.
test_datagen = ImageDataGenerator(rescale = 1./255)
# Membuat generator data untuk data uji
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/dataset/test_set',
                                            # mengubah ukuran gambar menjadi 64x64
                                            target_size = (64, 64),
                                            # memproses batch sebesar 32 gambar sekaligus
                                            batch_size = 32,
                                            # menggunakan mode kelas biner ('binary') karena kita mengasumsikan klasifikasi biner pada data uji.
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


Langkah 3 - Pembuatan Model CNN

Langkah 3.1.  - Inisiasi Model CNN

In [18]:
# Membuat objek model Sequential dari Keras
cnn = tf.keras.models.Sequential()

Langkah 3.2. - Pembuatan Layer Konvolusi 1

In [19]:
# menambahkan lapisan Convolutional Neural Network (CNN) ke model Sequential yang sudah dibuat sebelumnya
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Langkah 3.3 - Pembuatan Layer Pooling 1

In [20]:
# menambahkan lapisan MaxPooling ke model Convolutional Neural Network (CNN) yang sudah dibuat sebelumnya
# Lapisan MaxPooling ini membantu dalam mengekstraksi fitur-fitur utama dari hasil konvolusi dan mengurangi resolusi spatikal.
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Langkah 3.4 - Pembuatan Layer Konvolusi 2 dan Pooling 2

In [21]:
# menambahkan lapisan konvolusi diikuti dengan lapisan MaxPooling, model dapat secara bertahap mengekstraksi fitur-fitur penting dari gambar dan mereduksi dimensi spatialnya
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Langkah 3.5 - Flattening

In [22]:
# menambahkan lapisan Flatten
# mengubah representasi spasial tiga dimensi menjadi representasi satu dimensi sebelum masuk ke lapisan-lapisan fully connected (sepenuhnya terhubung) yang biasanya terdapat di bagian akhir model.
cnn.add(tf.keras.layers.Flatten())

Langkah 3.6 - Fully Connected Layer 1 (Input)

In [23]:
# menambahkan lapisan fully connected
# meningkatkan kapasitas representasional model, memungkinkannya untuk mempelajari pola-pola yang lebih kompleks dalam data
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Langkah 3.7 - Fully Connected Layer 2 (Output)

In [24]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Langkah 3.8 - Compile Model CNN

In [25]:
# Dengan lapisan Dense terakhir ini, model sekarang telah lengkap dan siap untuk dikompilasi (compile) sebelum dilakukan pelatihan (training)
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Langkah 4 - Fit CNN

In [28]:
# menggunakan metode fit untuk melatih (train) model Convolutional Neural Network (CNN) menggunakan data pelatihan (training_set) dan data validasi (test_set)
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
251/251 [==============================] - 45s 179ms/step - loss: 0.1828 - accuracy: 0.9261 - val_loss: 0.5984 - val_accuracy: 0.8005
Epoch 2/25
251/251 [==============================] - 38s 151ms/step - loss: 0.1780 - accuracy: 0.9295 - val_loss: 0.6009 - val_accuracy: 0.8050
Epoch 3/25
251/251 [==============================] - 38s 153ms/step - loss: 0.1676 - accuracy: 0.9345 - val_loss: 0.6406 - val_accuracy: 0.7955
Epoch 4/25
251/251 [==============================] - 38s 152ms/step - loss: 0.1617 - accuracy: 0.9350 - val_loss: 0.6076 - val_accuracy: 0.7900
Epoch 5/25
251/251 [==============================] - 38s 150ms/step - loss: 0.1548 - accuracy: 0.9409 - val_loss: 0.6384 - val_accuracy: 0.8010
Epoch 6/25
251/251 [==============================] - 39s 154ms/step - loss: 0.1417 - accuracy: 0.9474 - val_loss: 0.6901 - val_accuracy: 0.7950
Epoch 7/25
251/251 [==============================] - 43s 170ms/step - loss: 0.1402 - accuracy: 0.9428 - val_loss: 0.6672 - val_ac

Langkah 5 - Prediksi dengan 1 Citra

In [29]:
# Load dan Preprocess Gambar:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
# Mengubah gambar menjadi array NumPy
test_image = image.img_to_array(test_image)
# Menambahkan dimensi baru ke array untuk membuatnya sesuai dengan format yang diterima oleh model.
test_image = np.expand_dims(test_image, axis = 0)
# Melakukan Prediksi
result = cnn.predict(test_image)
training_set.class_indices
# Memeriksa hasil prediksi dari nilai yang diberikan oleh model
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 18ms/step
